In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing data set
house_data=pd.read_csv('../input/house-rent-prediction-for-hyderabad/hyd_v2.csv')

In [ ]:
house_data.head()

In [ ]:
new_data=house_data.drop(['amenities','locality','balconies','lift','active','loanAvailable','location','ownerName','parkingDesc','propertyTitle','propertyType','combineDescription','completeStreetName','facing','facingDesc','furnishingDesc','gym','id','isMaintenance','weight','waterSupply','swimmingPool','shortUrl','sharedAccomodation','reactivationSource'],axis=1)

In [ ]:
print(new_data.parking.value_counts())
print(new_data.bathroom.value_counts())
print(new_data.floor.value_counts())
print(new_data.type_bhk.value_counts())
print(new_data.maintenanceAmount.value_counts())



## encoding data

In [ ]:
new_data.replace({'parking':{'BOTH':0,'TWO_WHEELER':1,'FOUR_WHEELER':2,'NONE':3}},inplace=True)
new_data.replace({'type_bhk':{'RK1':0.5,'BHK1':1,'BHK2':2,'BHK3':3,'BHK4':4,'BHK4PLUS':5}},inplace=True)
new_data.replace({'maintenanceAmount':{'None':int(0)}},inplace=True)

In [ ]:
new_data.head()

In [ ]:
new_data.info()

In [ ]:
new_data.isnull().sum()

In [ ]:
hd=new_data.copy()

In [ ]:
hd['localityId'].value_counts()

In [ ]:
hd['maintenance']=hd['maintenanceAmount'].apply(lambda x:int(x))

In [ ]:
hd.head()

In [ ]:
hd1=hd.drop(['maintenanceAmount','deposit','property_age','totalFloor'],axis=1)

In [ ]:
hd1.head()

In [ ]:
hd1.info()

# Feature Engineering

In [ ]:
hd2=hd1.copy()
hd2['rent_per_sqft']=hd2['rent_amount']/hd2['property_size']
hd2.head()

In [ ]:
location_stats=hd2.groupby('localityId')['localityId'].agg('count').sort_values(ascending=False)
location_stats

In [ ]:
len(location_stats[location_stats<=10])

In [ ]:
location_less_than_10=location_stats[location_stats<=10]
location_less_than_10

In [ ]:
hd2.localityId=hd2.localityId.apply(lambda x:'NOT_FOUND' if x  in location_less_than_10 else x)
hd2['localityId'].value_counts()

## outliner detection

In [ ]:
hd2[hd2.property_size/hd2.type_bhk<300]

In [ ]:
hd2.shape

In [ ]:
hd3=hd2[~(hd2.property_size/hd2.type_bhk<300)]
hd3.shape

In [ ]:
hd3.rent_per_sqft.describe()

In [ ]:
def remove_pps_outliers (df):
    df_out = pd.DataFrame()
    for key,subdf in df.groupby('localityId'):
        m=np.mean(subdf.rent_per_sqft)
        st=np.std(subdf.rent_per_sqft) 
        reduced_df=subdf[(subdf.rent_per_sqft>(m-st))&(subdf.rent_per_sqft<=(m+st))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
hd4=remove_pps_outliers(hd3)
hd4.shape

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline


In [ ]:
mpl.rcParams['figure.figsize']=(20,10)
plt.hist(hd4.rent_per_sqft,rwidth=0.8)
plt.xlabel('price per sqft')
plt.ylabel('count')
plt.title('distribution of data')

In [ ]:
hd4.bathroom.unique()

In [ ]:
hd4[hd4.bathroom>hd4.type_bhk+1]

In [ ]:
hd5=hd4[hd4.bathroom<=hd4.type_bhk+1]

In [ ]:
hd5.shape

In [ ]:
hd6=hd5.drop('rent_per_sqft',axis=1)
hd6.head()

## one hot encoding

In [ ]:
dummies=pd.get_dummies(hd6.localityId)
dummies.head()

In [ ]:
hd7=pd.concat([hd6.drop('localityId',axis='columns'),dummies.drop('NOT_FOUND',axis='columns')],axis='columns')
hd7.head()

In [ ]:
X=hd7.drop('rent_amount',axis='columns')
Y=hd7['rent_amount']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,Y_train)
lr.score(X_test,Y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import cross_val_score 
cv = ShuffleSplit (n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression (),X,Y,cv=cv)

In [ ]:
X.columns

In [ ]:
def predict_price(localityId,bathroom,floor,parking,property_size,type_bhk,maintenance):

    loc_index = np.where(X.columns==localityId)[0][0] 
    x= np.zeros(len(X.columns))
    x[0]=bathroom
    x[1]=floor
    x[2]=parking
    x[3]=property_size
    x[4]=type_bhk
    x[5]=maintenance
    
    if loc_index>=0:
        x[loc_index]=1 

    return lr.predict([x])[0]



In [ ]:
predict_price('yella_reddy_guda_hyderabad',2,3,0,800,2,1000)

In [ ]:
predict_price('yella_reddy_guda_hyderabad',2,3,0,800,3,1000)